In [1]:
import numpy as np
import pandas as pd
import torch

In [8]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
val = pd.read_csv('data/val.csv')

In [16]:
X_train, y_train = train.drop('y',axis=1).to_numpy(), train['y'].to_numpy()
X_val, y_val = val.drop('y',axis=1).to_numpy(), val['y'].to_numpy()
X_test, y_test = test.drop('y',axis=1).to_numpy(), test['y'].to_numpy()


In [17]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
valid_data = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))
test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

# dataloaders
batch_size = 50

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [23]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print(f'Input size: {(sample_x.size())}') # batch_size, seq_length
print(f'Input: \n{sample_x}')
print()
print(f'Label size: {sample_y.size()}') # batch_size
print(f'Label: \n{sample_y}')

Input size: torch.Size([50, 2046])
Input: 
tensor([[    0,     0,     0,  ..., 61747, 34505, 36696],
        [    0,     0,     0,  ..., 58321, 27567, 36883],
        [    0,     0,     0,  ..., 19054, 39620, 35587],
        ...,
        [    0,     0,     0,  ..., 54244, 26679, 49121],
        [    0,     0,     0,  ..., 39620, 36633, 26679],
        [    0,     0,     0,  ..., 48776, 38184, 34841]])

Label size: torch.Size([50])
Label: 
tensor([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
        0, 1])
